# 06 - Pandas Metrics
All the metrics can also be queried from the Parquet files using a combination of DuckDB to generate the filtered joined timeseries, and Pandas to Generate the metrics.  They generate the exact same results, and so far DuckDB is faster in most (all) cases.  The Pandas version serves as a check.  Also, we anticipate that we may need to generate some statictcs that either can't be done or is prohibitive to do in SQL.

In [5]:
import teehr.queries.pandas as tqp
from pathlib import Path

In [6]:
CACHE_DIR = Path(Path.home(), "shared", "rti-eval")
STUDY_DIR = Path(CACHE_DIR, "post-event-example")
USGS = Path(STUDY_DIR, "timeseries/usgs/*.parquet")
MEDIUM_RANGE_MEM1 = Path(STUDY_DIR, "timeseries/medium_range_mem1/*.parquet")
SHORT_RANGE = Path(STUDY_DIR, "timeseries/short_range/*.parquet")
CROSSWALK = Path(STUDY_DIR, "geo/usgs_nwm22_crosswalk.parquet")
GEOMETRY = Path(STUDY_DIR, "geo/usgs_geometry.parquet")

In [10]:
%%time
metrics_df = tqp.get_metrics(
    primary_filepath=USGS,
    secondary_filepath=SHORT_RANGE,
    crosswalk_filepath=CROSSWALK,
    group_by=["primary_location_id", "reference_time"],
    order_by=["primary_location_id"],
    include_metrics=["bias"],
    filters=[
        {
            "column":  "primary_location_id",
            "operator": "in",
            "value": ["usgs-10336676","usgs-05129290", "usgs-05129515"]
        },
        {
            "column":  "reference_time",
            "operator": "=",
            "value": "2023-01-02 16:00:00"
        },
    ],
    geometry_filepath=GEOMETRY,
    include_geometry=True
)
metrics_df.transpose()

CPU times: user 801 ms, sys: 262 ms, total: 1.06 s
Wall time: 644 ms


,0,1,2
id,usgs-05129290,usgs-05129515,usgs-10336676
name,"GOLD PORTAGE OUTLET FROM KABETOGAMA LK NR RAY,MN",RAINY R AT BOAT LANDING BLW INTERNATIONAL FALL...,WARD C AT HWY 89 NR TAHOE PINES CA
geometry,POINT (-93.0748688 48.5243635),POINT (-93.4466667 48.5922222),POINT (-120.1576913 39.1321292)
primary_location_id,usgs-05129290,usgs-05129515,usgs-10336676
reference_time,2023-01-02 16:00:00,2023-01-02 16:00:00,2023-01-02 16:00:00
bias,1.030491,50.142329,0.792218
